In [1]:
from base_regression import *

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_regression
from sklearn.metrics import *


In [3]:
X, y = make_regression(1000, n_features=30, n_informative=5)

In [4]:
df = pd.read_csv("data/robotarm.csv")

In [5]:
df.head()

,l1,l2,theta1,theta2,phi,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,response
0,0.265509,0.802350,4.838564,5.908569,-0.728802,0.213364,0.111779,0.759629,0.173838,0.759032,...,0.704558,0.351085,0.675673,0.923004,0.175308,0.046382,0.082509,0.644592,0.787488,0.413735
1,0.372124,0.479495,4.337047,6.262863,-1.251046,0.969357,0.076628,0.523714,0.559547,0.324686,...,0.842908,0.687034,0.808810,0.691293,0.538147,0.514819,0.884079,0.187829,0.487828,0.221650
2,0.572853,0.177402,4.087068,1.463417,-1.151004,0.727771,0.172016,0.265373,0.918291,0.694882,...,0.589859,0.955572,0.427124,0.269862,0.034984,0.897163,0.528107,0.233737,0.768048,0.591990
3,0.908208,0.397133,0.469077,0.891591,0.940444,0.343964,0.842057,0.131919,0.178652,0.553256,...,0.437607,0.737775,0.709195,0.537626,0.825702,0.944048,0.633769,0.241314,0.769587,0.821492
4,0.201682,0.814227,5.671082,0.416556,-1.523702,0.144012,0.536417,0.373336,0.948005,0.016108,...,0.410825,0.368753,0.833490,0.277265,0.118353,0.967892,0.359782,0.667630,0.248911,0.369189


In [6]:
pred = [x for x in df.columns if x != 'response']
X = np.array(df[pred])
y = np.array(df[['response']]).flatten()

In [7]:
current_model = BMARS(X, 2)
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: 0.5058345391379547


0.50583453913795473

In [8]:
alpha, model1 = mh_iter(X, y, current_model)

action: birth
output: {'basis': [0, 9], 'sign': [-1, 1], 'knot': array([ 0.12028226,  0.00087225])}
acceptance_proba: 1.0
	bayes_factor: 1.0038054602338486
	prior_ratio: 1.184108123496835e-09
	proposal_ratio: 2533552418.45701


In [9]:
%%time
# run for 10 iters...
iters = 50

for idx in range(iters):
    try:
        alpha, next_model = mh_iter(X, y, current_model, False)
        current_model = BMARS(**next_model)   
    except:
        print("\n---\n")
        pass
    if idx % 10 == 0:
        eval_pipeline(current_model.construct_pipeline(False), X, y, verbose=True)
    

Result: 0.5101034815291323
Result: 0.5324070076610262
Result: 0.5305863721457605
Result: 0.5166576872438696
Result: 0.5148524836663725
Wall time: 17min 56s


In [10]:
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: 0.5074583383484548


0.50745833834845477

In [11]:
current_model.export()

{'X': array([[ 0.26550866,  0.80234954,  4.83856409,  5.90856851, -0.72880194,
          0.21336393,  0.11177927,  0.75962931,  0.173838  ,  0.75903228,
          0.41530902,  0.23547477,  0.82930183,  0.16968226,  0.72611049,
          0.94155787,  0.75035443,  0.80986214,  0.72848742,  0.70066313,
          0.73168672,  0.70455816,  0.35108534,  0.67567281,  0.92300411,
          0.17530818,  0.04638161,  0.08250872,  0.64459216,  0.78748828]]),
 'basis': [[0, 28],
  [24, 21],
  [10, 12],
  [6, 15],
  [3, 14],
  [17, 18],
  [2, 8],
  [11, 25],
  [5, 8],
  [8, 22],
  [0, 13],
  [18, 23],
  [24, 27],
  [5, 28],
  [17, 2],
  [2, 12],
  [19, 27],
  [1, 20],
  [0, 17],
  [19, 6],
  [7, 15]],
 'interaction': 2,
 'params': [{'knots': array([ 0.41330587,  0.62568204]), 'signs': [-1, 1]},
  {'knots': array([ 0.67961122,  0.39270235]), 'signs': [-1, 1]},
  {'knots': array([ 0.50513226,  0.57718178]), 'signs': [1, -1]},
  {'knots': array([ 0.33174715,  0.41551649]), 'signs': [1, -1]},
  {'knots